# Cirq implementation of Vernam cipher using QKD protocol KMB09s

#### Final Project, QXQ YLC 2024

Angel Martínez (anmartinezf@unal.edu.co)

Last update: April 20, 2024.

In [ ]:
!pip install cirq --quiet
import cirq
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 769.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

## Generating the key with QKD KMB09 protocol

In [ ]:
encode_gates = {0: cirq.I, 1: cirq.X}
basis_gates = {'Z': cirq.I, 'X': cirq.H}

num_bits = 200 #Number of qubits used for the generate the key (max length of the key)
qubits = cirq.NamedQubit.range(num_bits, prefix = 'q')

In [ ]:
alice_encode = random.choices([0, 1], k = num_bits) #Alice randomly chose their qubits initial state
alice_bases = random.choices(['Z', 'X'], k = num_bits)#Alice randomly chose her bases
print('\nAlice\'s randomly chosen bases: ', alice_bases[:10])

alice_circuit = cirq.Circuit()
for bit in range(num_bits):
  #Based on the choices using the dictionaries the corresponding gates are added to the circuit
  encode_value = alice_encode[bit]
  encode_gate = encode_gates[encode_value]

  basis_value = alice_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  alice_circuit.append(encode_gate(qubit))
  alice_circuit.append(basis_gate(qubit))

#Based on her bases a '0' or a '1' is added to the key
alice_key = [0 if base == 'Z' else 1 for base in alice_bases]
print('Alice\'s initial key: ', alice_key[:10]) #Printing the first ten values of alice's key

#The index of the i_th vector used of any base, it depends on the number of states N, in this case N=2
alice_index = [x + 1 for x in alice_encode]
print('Alice\'s key index: ', alice_index[:10])



Alice's randomly chosen bases:  ['Z', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'Z']
Alice's initial key:  [0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
Alice's key index:  [1, 2, 2, 1, 2, 1, 2, 1, 2, 1]


In [ ]:
bob_bases = random.choices(['Z', 'X'], k = num_bits)#Bob randomly chose her bases
print('Bob\'s randomly chosen bases: ', bob_bases[:10])

bob_circuit = cirq.Circuit()
for bit in range(num_bits):
  #Based on the choices using the dictionaries the corresponding gates are added to the circuit
  basis_value = bob_bases[bit]
  basis_gate = basis_gates[basis_value]

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit))

#Bob complete the circuit and measures
bob_circuit.append(cirq.measure(qubits, key = 'bob key'))
#print('\nBob\'s Phase 2 circuit:\n', bob_circuit)


kmb09_circuit = alice_circuit + bob_circuit
sim = cirq.Simulator()
results = sim.run(kmb09_circuit)
bob_measure = results.measurements['bob key'][0]

print('\nBob\'s measurements: ', bob_measure[:10])

#From their measurments and his chosen base he could obtain on of the four possible results
bob_result = ['0' if base == 'Z' and measure == 0 else '1' if base == 'Z' and measure == 1 else '+' if base == 'X' and measure == 0 else '-' for base, measure in zip(bob_bases, bob_measure)]
print('\nBob\'s result: ', bob_result[:10])

Bob's randomly chosen bases:  ['Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'X', 'Z']

Bob's measurements:  [0 0 0 0 0 0 1 0 1 0]

Bob's result:  ['0', '+', '+', '0', '+', '0', '1', '0', '-', '0']


In [ ]:
final_alice_key = []
final_bob_key = []

for bit in range(num_bits):
  #Using the table proposed in the original paper of KMB09 protocol we assigned a bit to the bob's key and save the respective value of alice's key
  if ((alice_index[bit] == 1 and bob_result[bit]=='1') or (alice_index[bit] == 2 and bob_result[bit]=='0')):
    final_bob_key.append(1)
    final_alice_key.append(alice_key[bit])

  elif((alice_index[bit] == 1  and bob_result[bit]=='-')  or (alice_index[bit] == 2 and bob_result[bit]=='+')):
    final_bob_key.append(0)
    final_alice_key.append(alice_key[bit])

  else: continue

print('\nAlice\'s key: ', final_alice_key[:10])
print('Bob\'s key: ', final_bob_key[:10])

if final_alice_key[0] == final_bob_key[0]: #Using one bit to check if there is a error
  final_alice_key = final_alice_key[1:]
  final_bob_key = final_bob_key[1:]

  print('\n\nWe can use our keys!')
  print('Alice Key: ', final_alice_key[:10])
  print('Bob Key: ', final_bob_key[:10])

else:
  print('\n\nEve was listening, we need to use a different channel!')


Alice's key:  [0, 0, 0, 0, 1, 1, 0, 1, 0, 1]
Bob's key:  [0, 0, 0, 0, 1, 1, 0, 1, 0, 1]


We can use our keys!
Alice Key:  [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
Bob Key:  [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]


In [ ]:
key = final_bob_key #Both keys are equal so we can use any of them to encrypt and decrypt the message
len(key) #The length of the key must be equal or greater that the message's length

43

## Using the key for send a message

In [ ]:
message = 'COLOMBIA'
print('\nMessage to be encrypted:',message)
key = ''.join(str(element) for element in key)
print('\nKey used to encrypt and decrypt:',key)
binary_map = { #Codification used to encript the message's symbols
  'A': '00000', 'B': '00001', 'C': '00010', 'D': '00011',
  'E': '00100', 'F': '00101', 'G': '00110', 'H': '00111',
  'I': '01000', 'J': '01001', 'K': '01010', 'L': '01011',
  'M': '01100', 'N': '01101', 'Ñ': '01110', 'O': '01111',
  'P': '10000', 'Q': '10001', 'R': '10010', 'S': '10011',
  'T': '10100', 'U': '10101', 'V': '10110', 'W': '10111',
  'X': '11000', 'Y': '11001', 'Z': '11010', ',': '11011',
  '?': '11100', '(': '11101', ')': '11110', '.': '11111'}
#Inverse dictionary to convert the encoding into the message
inverted_binary_map = {value: key for key, value in binary_map.items()}


Message to be encrypted: COLOMBIA

Key used to encrypt and decrypt: 0001101011000010111111100101010000100000001


In [ ]:
def encrypt(key, message):
  """
  Performs an Vernam Cipher in a given message returnuing the resulting encrypted message
    given a key. To make this a XOR operation is performed bit by bit between each pair of
    characters, where each character is first converted to its binary representation.

  Parameters
  ----------
  key : str
      The key used to encrypt the message. It must have the same length as the message.
  message : str
      The message to be encrypted.

  Returns
  -------
  str
      The resulting encrypted message, represented as a string.

  Raises
  ------
  IndexError
      If the length of the key length is less that the length of the message.

  Example
  --------
  >>> encrypt('key', 'message')
  'encrypted_message'
  """
  binary = ''.join(binary_map[letter.upper()] for letter in message)#Converts the word on the corresponding codes usig the binary_map
  encrypted_msg = ""
  for i in range(len(binary)):
    current_xor = str(int(binary[i]) ^ int(key[i]))#Apply XOR operation on the message in binary and the key
    encrypted_msg += current_xor #Add the result to the returned variable
  return encrypted_msg

In [ ]:
def decrypt(key, encrypted_msg):
  """
  Performs a decryption on a given encrypted message, returning the original message.
  This is achieved by performing a bitwise XOR operation between each character of the
  encrypted message and its corresponding character in the key, where each character
  is first converted to its binary representation.

  Parameters
  ----------
  key : str
      The key used to decrypt the message. It must have the same length as the encrypted message.
  encrypted_msg : str
      The encrypted message to be decrypted.

  Returns
  -------
  list
      The original message, represented as a list of characters.

  Raises
  ------
  IndexError
      If the length of the key does not match the length of the encrypted message.

  Example
  --------
  >>> decrypt('key', 'encrypted_message')
  ['H', 'E', 'L', 'L', 'O']
  """
  result = ""
  for i in range(len(encrypted_msg)):
      current_xor = str(int(encrypted_msg[i]) ^ int(key[i]))#Apply XOR operation on the encripted binary message and the key to decrypt
      result += current_xor#Add the result to a decrypted binary code

  decrypted_msg = []
  for i in range(0, len(result), 5):
      code = result[i:i+5]#Divide the binary code into groups that corresponding to the symbols of the binary map
      decrypted_msg.append(inverted_binary_map[code])#Add each letter of the decrypted message

  return decrypted_msg

Testing the encrypt and decrypt functions with the key obtained using KMB09 protocol and the chosen message.

In [ ]:
encrypted_msg = encrypt(key,message)
encrypted_msg

'0000100100010100000010000101000100100000'

In [ ]:
decrypted_msg = decrypt(key, encrypted_msg)
decrypted_msg

['C', 'O', 'L', 'O', 'M', 'B', 'I', 'A']

References
==========
1.  The Coding School \"Homework 9: Ιmplementing BB84 - Part I.\", (2024).
2. M. Khan, M. Murphy, A. Beige "High error-rate quantum key distribution for long-distance
communication", [arXiv:0901.3909](https://arxiv.org/abs/0901.3909), (2009).
3. C. E. Shannon "Communication Theory of Secrecy Systems", Bell Syst. Tech. J., vol. 28, no.
4, pp. 656-715, (1949).
4. G. S. Vernam "Cipher Printing Telegraph Systems For Secret Wire and Radio Telegraphic
Communications", J. Amer. Inst. Elec. Eng., vol. 55, no. 2, pp. 109-115, (1926).

